In [1]:
def clearall():
    tall = [var for var in globals() if var != ['']]
    for var in tall:
        del globals()[var]

In [2]:
print('Importing Librairies...')
import warnings
warnings.filterwarnings("ignore")
import talib
import numpy as np
import pandas as pd
import pandas_datareader as web
from colorama import Fore, Back, Style
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from sklearn.preprocessing import MinMaxScaler
import yaml
from keras.models import model_from_yaml
from sklearn.metrics import  precision_score,accuracy_score,recall_score

import sys

import xgboost as xgb
from xgboost import XGBRegressor, plot_importance
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn')
import time
import datetime as dt
import os
import shutil
import ib_insync
from ib_insync import *

import smtplib
import mimetypes
from email.message import EmailMessage
yaml.warnings({'YAMLLoadWarning': False})
warnings.filterwarnings("ignore")
print('Librairies imported')
print('')

Importing Librairies...


Using TensorFlow backend.


Librairies imported



In [179]:
start = '2009-01-01'
end='2019-12-04'

LeChemin = 'DONNEES_DL/'
list_dir = os.listdir(LeChemin+'.')

NEW_LIST = []
for pikunichihouahoua in range(0,len(str(list_dir).split(','))):
    if str(list_dir).split(',')[pikunichihouahoua].split(',')[0].split('.')[1].split("'")[0]=='yaml':
        NEW_LIST.append(str(list_dir).split(',')[pikunichihouahoua].split(',')[0].split('.')[0].split("'")[1].split('_')[1])
NEW_LIST = sorted(set(NEW_LIST))

In [4]:
print(' - Date de début : ',start,' - Date de fin : ',end)
print(Fore.YELLOW,'GENERATION DE LaBase :',Fore.MAGENTA,time.strftime("%H:%M:%S", time.localtime()))

LaBase = pd.DataFrame()
tmps55=time.time()
for ticker in NEW_LIST:
    
    df = web.DataReader(ticker,'yahoo',start,end)
    df['Symbol'] = ticker
    LaBase = pd.concat((LaBase,df))
    tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"\n Time for completing LaBase Generation = %f" %tmps22,'seconds \n',Style.RESET_ALL)
print('Taille df : ',LaBase.shape)

    
#LaBase['Date'] = LaBase.index
#LaBase['Date'] = LaBase['Date'].dt.strftime('%Y-%m-%d')

LaBase = LaBase.drop(['Close'],axis=1)
LaBase['Close'] = LaBase['Adj Close']
LaBase = LaBase.drop(['Adj Close'],axis = 1)

LaBase = LaBase[['Date','Symbol','High','Low','Open','Volume','Close']] 
LaBase.to_csv('LaBase.csv')

 - Date de début :  2009-01-01  - Date de fin :  2019-12-04
 GENERATION DE LaBase :  17:25:18
 
 Time for completing LaBase Generation = 501.210000 seconds 
 
Taille df :  (895193, 7)


In [14]:
LaBase.head()

,Symbol,High,Low,Open,Volume,Close
Date,,,,,,
2009-01-02,KSU,20.760000,19.000000,19.020000,1723500.0,18.548553
2009-01-02,HOG,18.570000,16.980000,17.250000,3592500.0,14.442327
2009-01-02,XRX,22.187088,20.764164,21.001318,1709100.0,18.928757
2009-01-02,PNR,16.702484,15.829415,16.057756,1154700.0,13.246778
2009-01-02,BLL,10.745000,10.372500,10.560000,2949200.0,8.769405


In [21]:
LaBase.tail()

,Symbol,High,Low,Open,Volume,Close
Date,,,,,,
2019-12-04,AXP,118.540001,117.000000,117.309998,2444100.0,117.940002
2019-12-04,NVDA,211.919998,208.270004,211.020004,6835600.0,209.419998
2019-12-04,XOM,68.760002,68.139999,68.250000,11049700.0,68.650002
2019-12-04,ICE,94.120003,93.089996,93.360001,2260400.0,93.320000
2019-12-04,BIIB,293.140015,284.029999,290.149994,1736200.0,289.519989


In [20]:
%%time
LaBase.to_csv('LaBase.csv')

CPU times: user 12.1 s, sys: 174 ms, total: 12.3 s
Wall time: 12.4 s


In [22]:
LaBase.shape

(895193, 6)

In [23]:
#### DEF ####

def boost(df):
    X = df
    X = X.drop(['Close'],axis=1)
    X['Close'] = df['Close']
    y = X.iloc[:,-1]
    Xtrain = X.iloc[:-2,:-1]
    Xtest = X.iloc[-2:-1,:-1]
    yshift = y.shift(-1)
    ytrain = yshift.iloc[:-2]
    ytest = yshift.iloc[-2:-1]

    model = xgb.XGBRegressor(n_estimators=20000, objective='reg:squarederror',learning_rate=1, gamma=1, subsample=1, colsample_bytree=1, max_depth=100)

    model.fit( Xtrain, ytrain, early_stopping_rounds=150, eval_set=[(Xtest, ytest)], verbose=0)

    ytrain_pred = model.predict(Xtrain)

    y_pred = model.predict(Xtest)

    pred = model.predict(X.iloc[:,:-1])

    df['Close.S'] = pred
    df['Close.S2'] = df['Close.S']
    df = df.dropna()
    return(df)

def prepa_data(df):
    rsi = talib.RSI(df['Close'],timeperiod=14)
    stoc_slowk, stoc_slowd = talib.STOCH(df['High'],df['Low'],df['Close'])
    upper, middle, lower =  talib.BBANDS(df['Close'], timeperiod=9, nbdevup=2, nbdevdn=2,matype=0)
    sma5 = talib.SMA(df['Close'],timeperiod=5)
    sma8 = talib.SMA(df['Close'],timeperiod=8)
    sma10 = talib.SMA(df['Close'],timeperiod=10)
    sma12 = talib.SMA(df['Close'],timeperiod=12)
    sma15 = talib.SMA(df['Close'],timeperiod=15)
    sma30 = talib.SMA(df['Close'],timeperiod=30)
    sma35 = talib.SMA(df['Close'],timeperiod=35)
    sma40 = talib.SMA(df['Close'],timeperiod=40)
    sma45 = talib.SMA(df['Close'],timeperiod=45)
    sma50 = talib.SMA(df['Close'],timeperiod=50)
    atr = talib.ATR(df['High'],df['Low'],df['Close'],timeperiod=10)
    delta5_8 = sma5 - sma8
    delta8_10 = sma8 - sma10
    delta10_12 = sma10 - sma12
    delta12_15 = sma12 - sma15
    delta15_30 = sma15 - sma30
    delta30_35 = sma30 - sma35
    delta35_40 = sma35 - sma40
    delta40_45 = sma40 - sma45
    delta45_50 = sma45 - sma50
    bbdelta = upper - middle
    price_bolup = df['Close'] - lower
    price_bolow = df['Close'] - upper
    Ema = talib.EMA(df['Close'],timeperiod=20)
    KC_High = Ema + 2*atr
    KC_Low = Ema - 2*atr
    aroondown, aroonup = talib.AROON(df['High'], df['Low'], timeperiod=9)
    aroon = aroonup - aroondown 
    rsi30_list = []
    rsi70_list = []
    for i in range(0,df.shape[0]):
        
        rsi70_list.append(70 - rsi.iloc[i])
        rsi30_list.append(rsi.iloc[i] - 30)
        
    varop_spy = df['Open'] - df['Close']
    varhl_spy = df['High'] - df['Low']
    df['Varop_Spy'] = varop_spy
    df['Varhl_spy'] = varhl_spy
    df['RSI'] = rsi
    df['70 - RSI'] = np.array(rsi70_list)
    df['RSI - 30'] = np.array(rsi30_list)
    df['BBD_Delta_Up'] = bbdelta
    df['delta5_8'] = delta5_8
    df['delta8_10'] = delta8_10
    df['delta10_12'] = delta10_12
    df['delta12_15'] = delta12_15
    df['delta15_30'] = delta15_30
    df['delta30_35'] = delta30_35
    df['delta35_40'] = delta35_40
    df['delta40_45'] = delta40_45
    df['delta45_50'] = delta45_50
    df['Stoc_Slowk'] = stoc_slowk
    df['Stoc_Slowd'] = stoc_slowd
    df['KC_High'] = KC_High
    df['KC_Low'] = KC_Low
    df['upper'] = upper
    df['lower'] = lower
    df['var_bollup_kchigh'] = upper-KC_High
    df['var_bolllow_kclow'] = lower-KC_Low
    df['Aroon Up'] = aroonup
    df['Aroon Down'] = aroondown
    df['Delta Aroon'] = aroon
    up = []
    down = []
    df = df.dropna()
    df = boost(df)
    df['%Futur'] = ((df['Close.S']-df['Close']) *100) / (df['Close'])
    df['%Futur2'] = ((df['Close.S2']-df['Close']) *100) / (df['Close'])
    for i in range(0,df.shape[0]-5):
        if df.iloc[i]['%Futur'] > 0.5 :
            up.append(1)
            down.append(0)
        elif df.iloc[i]['%Futur'] < -0.5:
            up.append(0)
            down.append(1)
        else:
            up.append(0)
            down.append(0)
    up.append(0)
    down.append(0)
    up.append(0)
    down.append(0)
    up.append(0)
    down.append(0)
    up.append(0)
    down.append(0)
    up.append(0)
    down.append(0)

    df['target_up'] = up
    df['target_down'] = down
    return(df)

def deep_learning(df):
    X = df.copy()
    #####################X = X.iloc[:,1:-4]
    X = X.iloc[:,:-4]
    X.astype(np.float64)
    y_up = df.iloc[:,-2].values
    y_down = df.iloc[:,-1].values

    scaler = MinMaxScaler(feature_range=(0, 1))
    X = scaler.fit_transform(X)
    y_up = np.array(y_up).reshape(-1,1)
    y_down = np.array(y_down).reshape(-1,1)

    Xtrain = X[:bloc1,:]
    Xtest = X[bloc1:,:]
    ytrain_up = y_up[:bloc1,:]
    ytest_up = y_up[bloc1:,:]
    ytrain_down = y_down[:bloc1,:]
    ytest_down = y_down[bloc1:,:]

    seed = 770
    np.random.seed(seed)

    ytrain_up = ytrain_up.reshape(ytrain_up.shape[0],)
    ytrain_down = ytrain_down.reshape(ytrain_down.shape[0],)

    Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1])

    yhat_up = model_up.predict_classes(Xtest)
    yhat_down = model_down.predict_classes(Xtest)

    predict_up = model_up.predict(Xtest)
    predict_down = model_down.predict(Xtest)

    precision_up = precision_score(ytest_up, yhat_up)  
    precision_down = precision_score(ytest_down, yhat_down) 

    resultats = pd.DataFrame()
    resultats['Date'] = df.index[bloc1:]
    resultats.index= df.index[bloc1:]
    resultats['Move Up'] = yhat_up
    resultats['Confiance up'] = (predict_up)*100
    resultats['Move Down'] = yhat_down
    resultats['Confiance Down'] = (predict_down)*100
    resultats['Actual'] = df.iloc[bloc1:]['Close']
    resultats['Actual.S'] = df.iloc[bloc1:]['Close.S']
    open_S = df['Open'].shift(-1)
    resultats['Open.S'] = open_S.iloc[bloc1:]
    dmp_cp=[]
    dmp_cp = ((resultats['Confiance up']-resultats['Confiance Down'])/(resultats['Confiance up']+resultats['Confiance Down'])*100)
    resultats['DMP_CP'] = dmp_cp
    
    return(resultats,precision_up,precision_down,scaler)

In [28]:
###### PREPARATION #########


print('The computing has begun at ',dt.datetime.now().hour,':',dt.datetime.now().minute,':',dt.datetime.now().second)
print('Le denier signal est ',Fore.YELLOW,LaBase.index[-1],Style.RESET_ALL)

LaBasePrep = pd.DataFrame()
tmps55=time.time()
for loop,ticker in enumerate(NEW_LIST):

    df = LaBase[LaBase['Symbol']==ticker]
    df = df.drop(['Symbol'],axis=1)
    
    df = prepa_data(df)
    df['Symbol'] = ticker
    LaBasePrep = pd.concat((LaBasePrep,df))
tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"Time for complete Signals Generation = %f" %tmps22,'seconds \n',Style.RESET_ALL)
LaBasePrep.to_csv('LaBasePrep.csv')

The computing has begun at  22 : 30 : 43
Le denier signal est   2019-12-04 00:00:00 
 Time for complete Signals Generation = 3385.090000 seconds 
 


In [30]:
LaBasePrep.shape,ticker

((878680, 38), 'BIIB')

In [34]:
len(NEW_LIST) ,len(sorted(LaBasePrep['Symbol'].unique()))

(337, 337)

In [35]:
LaBasePrep.head()

,High,Low,Open,Volume,Close,Varop_Spy,Varhl_spy,RSI,70 - RSI,RSI - 30,...,Aroon Up,Aroon Down,Delta Aroon,Close.S,Close.S2,%Futur,%Futur2,target_up,target_down,Symbol
Date,,,,,,,,,,,,,,,,,,,,,
2009-03-16,15.59,14.50,14.81,2938500.0,13.372422,1.437578,1.09,38.356642,31.643358,8.356642,...,0.000000,33.333333,-33.333333,13.684612,13.684612,2.334581,2.334581,1,0,KSU
2009-03-17,15.06,14.25,14.64,2122400.0,13.572555,1.067446,0.81,40.153464,29.846536,10.153464,...,88.888889,22.222222,66.666667,13.581218,13.581218,0.063829,0.063829,0,0,KSU
2009-03-18,15.23,14.41,14.90,3182200.0,13.672623,1.227377,0.82,41.078283,28.921717,11.078283,...,77.777778,11.111111,66.666667,13.435838,13.435838,-1.731818,-1.731818,0,1,KSU
2009-03-19,15.22,14.63,15.17,1945500.0,13.381522,1.788478,0.59,39.181444,30.818556,9.181444,...,66.666667,0.000000,66.666667,12.156712,12.156712,-9.152999,-9.152999,0,1,KSU
2009-03-20,14.77,13.47,14.77,2001100.0,12.298994,2.471006,1.30,33.066560,36.933440,3.066560,...,55.555556,0.000000,55.555556,13.315935,13.315935,8.268490,8.268490,1,0,KSU


In [297]:
LaBasePrep.tail()

,High,Low,Open,Volume,Close,Varop_Spy,Varhl_spy,RSI,70 - RSI,RSI - 30,...,Aroon Up,Aroon Down,Delta Aroon,Close.S,Close.S2,%Futur,%Futur2,target_up,target_down,Symbol
Date,,,,,,,,,,,,,,,,,,,,,
2019-11-27,304.000000,298.679993,303.609985,1369900.0,303.350006,0.259979,5.320007,69.816598,0.183402,39.816598,...,88.888889,0.000000,88.888889,298.718445,298.718445,-1.526804,-1.526804,0,0,BIIB
2019-11-29,305.179993,299.399994,301.100006,944600.0,299.809998,1.290009,5.779999,65.256142,4.743858,35.256142,...,77.777778,0.000000,77.777778,293.587769,293.587769,-2.075391,-2.075391,0,0,BIIB
2019-12-02,296.600006,290.160004,292.350006,1822200.0,292.390015,-0.040009,6.440002,56.870775,13.129225,26.870775,...,66.666667,0.000000,66.666667,289.342957,289.342957,-1.042121,-1.042121,0,0,BIIB
2019-12-03,294.649994,289.799988,291.779999,1644900.0,290.700012,1.079987,4.850006,55.133047,14.866953,25.133047,...,55.555556,0.000000,55.555556,288.205688,288.205688,-0.858040,-0.858040,0,0,BIIB
2019-12-04,293.140015,284.029999,290.149994,1736200.0,289.519989,0.630005,9.110016,53.894744,16.105256,23.894744,...,44.444444,11.111111,33.333333,285.571960,285.571960,-1.363646,-1.363646,0,0,BIIB


In [48]:
len(LaBasePrep[LaBasePrep['Symbol']==ticker].index.unique()),len(LaBasePrep.index.unique())

(2701, 2701)

In [295]:
LaBasePrep['Date'] = LaBasePrep.index
LaBasePrep['Date'] = LaBasePrep['Date'].dt.strftime('%Y-%m-%d')
LaBasePrep.index = pd.to_datetime(LaBasePrep['Date'],format='%Y-%m-%d')
LaBasePrep = LaBasePrep.drop(['Date'],axis=1)

In [296]:
LaBasePrep.index[819]

Timestamp('2012-06-13 00:00:00')

In [71]:
# FAST & PRU => 2019-12-04

In [322]:
%%time
ALMANACH = sorted(LaBasePrep.index.strftime('%Y-%m-%d').unique())

CPU times: user 5.03 s, sys: 48.6 ms, total: 5.08 s
Wall time: 5.07 s


In [ ]:
tmps55=time.time()
BaseDL = pd.DataFrame()

yamlup = ('DONNEES_DL/Save_'+ticker+'_up.yaml')
yamldown = ('DONNEES_DL/Save_'+ticker+'_down.yaml')
modelup = ('DONNEES_DL/Save_'+ticker+'_up.h5')
modeldown = ('DONNEES_DL/Save_'+ticker+'_down.h5')

yaml_file_up = open(yamlup, 'r')
yaml_file_down = open(yamldown, 'r')
model_yaml_up = yaml_file_up.read()
model_yaml_down = yaml_file_down.read()
yaml_file_up.close()
yaml_file_down.close()
model_up = model_from_yaml(model_yaml_up)
model_down = model_from_yaml(model_yaml_down)
# load weights into new model
model_up.load_weights(modelup)
model_down.load_weights(modeldown)
GREEN = []
RED = []
GREEN_CLOSE = []
RED_CLOSE=[]
TICKER = []
DATE = []
PRICE = []
i=-1
for ticker in NEW_LIST:
    for elem in ALMANACH:
        df = LaBasePrep[LaBasePrep['Symbol']==ticker]
        df = df[df.index<=elem]
        if len(df)<820:
            continue
        df = df.iloc[-820:,:]
        df = df.drop(['Symbol'],axis=1)
        bloc1 = 616

        resultats,precision_up,precision_down,scaler = deep_learning(df)
        price = df.iloc[i]['Close']

        if (precision_up * 100) > 69 and (precision_down * 100) > 69:

            ####################
            ##### SIGNALS #####
            ###################

            filtre_up = 93
            filtre_down = 97
            filtre_up_close = 80
            filtre_down_close = 80

            if resultats.iloc[i]['Confiance Down'] > filtre_down:
                doob = -1
            elif resultats.iloc[i]['Confiance Down'] > filtre_down_close and resultats.iloc[i]['Confiance Down'] < filtre_down :
                waab = -1
                doob = 0
            elif resultats.iloc[i]['Confiance up'] > filtre_up:
                doob = 1
            elif resultats.iloc[i]['Confiance up'] > filtre_up_close and resultats.iloc[i]['Confiance up'] < filtre_up:
                waab = 1
                doob = 0
            else :
                doob = 0
                waab = 0

            if doob == 1 :
                DATE.append(df.index[i])
                TICKER.append(ticker)
                GREEN.append(1)
                RED.append(0)
                GREEN_CLOSE.append(0)
                RED_CLOSE.append(1)
                PRICE.append(price)

            elif doob == -1 :
                DATE.append(df.index[i])
                TICKER.append(ticker)
                GREEN.append(0)
                RED.append(1)
                GREEN_CLOSE.append(1)
                RED_CLOSE.append(0)
                PRICE.append(price)

            elif doob == 0 and waab == -1 :
                DATE.append(df.index[i])
                TICKER.append(ticker)
                GREEN.append(0)
                RED.append(0)
                GREEN_CLOSE.append(1)
                RED_CLOSE.append(0)
                PRICE.append(price)

            elif doob == 0 and waab == 1 :
                DATE.append(df.index[i])
                TICKER.append(ticker)
                GREEN.append(0)
                RED.append(0)
                GREEN_CLOSE.append(0)
                RED_CLOSE.append(1)
                PRICE.append(price)

            else:
                DATE.append(df.index[i])
                TICKER.append(ticker)
                GREEN.append(0)
                RED.append(0)
                GREEN_CLOSE.append(0)
                RED_CLOSE.append(0)
                PRICE.append(price)

        else:
            DATE.append(df.index[i])
            TICKER.append(ticker)
            GREEN.append(0)
            RED.append(0)
            GREEN_CLOSE.append(0)
            RED_CLOSE.append(0)
            PRICE.append(price)
signals = pd.DataFrame(columns = ['Date','Ticker','Open Long','Close Long','Open Short','Close Short'])            
signals['Date'] = DATE
signals['Ticker'] = TICKER
signals['Open Long'] = GREEN
signals['Close Long'] = GREEN_CLOSE
signals['Open Short'] = RED
signals['Close Short'] = RED_CLOSE 
signals['Close Price'] = PRICE
tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"Time for complete DL Generation = %f" %tmps22,'seconds \n',Style.RESET_ALL)

In [192]:
sum(GREEN),sum(GREEN_CLOSE),sum(RED),sum(RED_CLOSE)

(31907, 31456, 22396, 37480)

In [193]:
signals

,Date,Ticker,Open Long,Close Long,Open Short,Close Short,Close Price
0,2019-12-04,A,0,0,0,0,80.930000
1,2019-12-03,A,0,0,0,0,80.099998
2,2019-12-02,A,0,0,0,0,80.349998
3,2019-11-29,A,0,0,0,0,80.769997
4,2019-11-27,A,0,0,0,0,81.080002
...,...,...,...,...,...,...,...
602838,2012-06-19,ZION,0,0,0,0,17.863342
602839,2012-06-18,ZION,0,0,0,0,17.285324
602840,2012-06-15,ZION,0,1,1,0,17.156883
602841,2012-06-14,ZION,0,0,0,0,16.844944


In [194]:
signals.to_csv('SIGNALS_10_ANS.csv')

In [284]:
df1 = pd.read_csv('SIGNALS_2019-11-29.csv')
df1 = df1.drop(['Unnamed: 0'],axis=1)
df1 = df1.sort_values(['Ticker'])

In [285]:
df1

,Date,Ticker,Open Long,Close Long,Open Short,Close Short,Close Price
1,2019-12-03,ALK,1,0,0,1,66.610001
30,2019-12-03,APA,1,0,0,1,18.379999
28,2019-12-03,CI,1,0,0,1,195.330002
7,2019-12-03,CNC,0,1,1,0,60.070000
2,2019-12-03,COG,0,0,0,1,15.710000
21,2019-12-03,CTSH,1,0,0,1,62.750000
12,2019-12-03,CXO,0,1,1,0,71.070000
9,2019-12-03,DAL,0,1,1,0,55.619999
8,2019-12-03,HCA,1,0,0,1,138.169998
26,2019-12-03,HES,0,1,1,0,59.790001


In [287]:
df2 = signals[signals['Date']=='2019-11-29']

In [288]:
df2['Date'] = pd.to_datetime(df2['Date'],format='%Y-%m-%d')
df2 = df2.sort_values(['Ticker'])

In [289]:
df2 = df2[(df2['Close Long']==1)|(df2['Close Short']==1)]

In [290]:
df2

,Date,Ticker,Open Long,Close Long,Open Short,Close Short,Close Price
13177,2019-11-29,ADP,0,1,0,0,170.779999
30115,2019-11-29,AMAT,0,1,1,0,57.900002
41076,2019-11-29,ANSS,0,1,1,0,254.690002
190193,2019-11-29,EQIX,0,1,1,0,566.849976
195839,2019-11-29,ETN,0,1,1,0,92.500000
219701,2019-11-29,FISV,0,1,1,0,116.239998
238033,2019-11-29,GL,0,1,1,0,102.760002
259593,2019-11-29,HFC,1,0,0,1,51.549999
333268,2019-11-29,LEN,0,1,1,0,59.650002
390655,2019-11-29,MRO,1,0,0,1,11.650000


In [291]:
print('Longueur des df1 & df2 :', df1.shape[0],' & ',df2.shape[0],' = ',abs(df1.shape[0]-df2.shape[0]),len(pd.concat([df1, df2]).loc[df1.index.symmetric_difference(df2.index)]))
pd.concat([df1, df2]).loc[df1.index.symmetric_difference(df2.index)]

Longueur des df1 & df2 : 32  &  19  =  13 51


,Date,Ticker,Open Long,Close Long,Open Short,Close Short,Close Price
0,2019-12-03,XYL,0,1,1,0,75.540001
1,2019-12-03,ALK,1,0,0,1,66.610001
2,2019-12-03,COG,0,0,0,1,15.710000
3,2019-12-03,XOM,1,0,0,1,67.879997
4,2019-12-03,PNR,1,0,0,1,43.619999
5,2019-12-03,LOW,1,0,0,1,114.550003
6,2019-12-03,NWS,0,1,1,0,12.910000
7,2019-12-03,CNC,0,1,1,0,60.070000
8,2019-12-03,HCA,1,0,0,1,138.169998
9,2019-12-03,DAL,0,1,1,0,55.619999


In [239]:
df2

,Date,Ticker,Open Long,Close Long,Open Short,Close Short,Close Price
13174,2019-12-04,ADP,0,1,1,0,168.570007
30112,2019-12-04,AMAT,0,1,0,0,56.090000
52365,2019-12-04,ATO,0,1,0,0,106.989998
67272,2019-12-04,BBY,0,1,1,0,79.180000
69154,2019-12-04,BDX,0,1,1,0,257.230011
100784,2019-12-04,CCL,0,1,1,0,43.880001
160078,2019-12-04,DISCA,1,0,0,1,32.200001
182662,2019-12-04,EL,0,1,1,0,197.869995
190190,2019-12-04,EQIX,0,1,1,0,562.929993
195836,2019-12-04,ETN,0,1,1,0,91.120003


In [240]:
df1

,Date,Ticker,Open Long,Close Long,Open Short,Close Short,Close Price
8,2019-12-04,CCL,0,1,0,0,43.880001
19,2019-12-04,CHTR,1,0,0,1,462.049988
7,2019-12-04,DAL,0,1,1,0,56.060001
10,2019-12-04,DISCK,0,1,1,0,29.930000
14,2019-12-04,DLR,0,1,1,0,119.099998
25,2019-12-04,FANG,1,0,0,1,80.559998
0,2019-12-04,FAST,0,1,1,0,35.060001
6,2019-12-04,FDX,1,0,0,1,153.869995
21,2019-12-04,FLS,1,0,0,1,46.290001
11,2019-12-04,FTV,1,0,0,1,71.410004


In [300]:
signals[(signals['Close Long']==1)|(signals['Close Short']==1)|(signals['Open Long']==1)|(signals['Open Short']==1)]

,Date,Ticker,Open Long,Close Long,Open Short,Close Short,Close Price
406,2018-04-25,A,1,0,0,1,64.924988
462,2018-02-02,A,0,1,0,0,70.145943
636,2017-05-25,A,0,1,1,0,58.003189
637,2017-05-24,A,0,1,1,0,57.934727
653,2017-05-02,A,0,1,1,0,54.706886
...,...,...,...,...,...,...,...
602826,2012-07-06,ZION,0,1,1,0,17.789942
602829,2012-07-02,ZION,0,1,0,0,18.010139
602832,2012-06-27,ZION,0,1,0,0,17.615625
602835,2012-06-22,ZION,0,1,0,0,17.551395


In [294]:
sum(GREEN),sum(GREEN_CLOSE),sum(RED),sum(RED_CLOSE)

(31907, 31456, 22396, 37480)